In [13]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [14]:
# ==================
# Constant
# ==================
ex = "003"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [15]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",]

In [16]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [17]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 10:46:17,630 - INFO - logger set up


<RootLogger root (DEBUG)>

In [18]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [20]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [21]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.335846	valid_1's rmse: 0.420878
[400]	training's rmse: 0.277013	valid_1's rmse: 0.420366


2021-08-17 10:46:27,941 - INFO - Fold0:CV=0.4189932073029282
2021-08-17 10:46:27,955 - INFO - [fold 0] done in 3 s


Early stopping, best iteration is:
[319]	training's rmse: 0.297456	valid_1's rmse: 0.418993
Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.334007	valid_1's rmse: 0.426545
[400]	training's rmse: 0.275677	valid_1's rmse: 0.422627
[600]	training's rmse: 0.237548	valid_1's rmse: 0.423552


2021-08-17 10:46:31,386 - INFO - Fold1:CV=0.42191543311354185
2021-08-17 10:46:31,404 - INFO - [fold 1] done in 3 s


Early stopping, best iteration is:
[459]	training's rmse: 0.26312	valid_1's rmse: 0.421915
Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.329886	valid_1's rmse: 0.454806
[400]	training's rmse: 0.272843	valid_1's rmse: 0.453117


2021-08-17 10:46:33,782 - INFO - Fold2:CV=0.452645288509369
2021-08-17 10:46:33,800 - INFO - [fold 2] done in 2 s


Early stopping, best iteration is:
[293]	training's rmse: 0.2992	valid_1's rmse: 0.452645
Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.33329	valid_1's rmse: 0.432553
[400]	training's rmse: 0.27424	valid_1's rmse: 0.432774


2021-08-17 10:46:36,125 - INFO - Fold3:CV=0.43192341858256966
2021-08-17 10:46:36,140 - INFO - [fold 3] done in 2 s


Early stopping, best iteration is:
[299]	training's rmse: 0.300631	valid_1's rmse: 0.431923
Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.329549	valid_1's rmse: 0.454169
[400]	training's rmse: 0.272324	valid_1's rmse: 0.449875
[600]	training's rmse: 0.233916	valid_1's rmse: 0.451823
Early stopping, best iteration is:
[419]	training's rmse: 0.267952	valid_1's rmse: 0.449252


2021-08-17 10:46:38,994 - INFO - Fold4:CV=0.4492522271645769
2021-08-17 10:46:39,010 - INFO - [fold 4] done in 3 s
2021-08-17 10:46:39,022 - INFO - CV=0.43516458973760686
2021-08-17 10:46:39,026 - INFO - [lightgbm] done in 14 s


In [22]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [23]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [24]:
sub

,MAL_ID,Score
0,5,8.137468
1,6,8.285497
2,7,7.486237
3,8,6.620687
4,15,7.368244
...,...,...
7416,47307,6.846042
7417,47398,7.434997
7418,47402,7.262702
7419,47616,6.915645
